In [ ]:
%pip install optuna
%pip install lightgbm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.1 MB/s eta 0:00:00


# Lab 2

In [ ]:
import optuna
import warnings
import numpy as np
import pandas as pd
import lightgbm as lgb

from google.colab import drive
from numba import njit, prange
from itertools import combinations
from optuna.samplers import TPESampler

from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GroupKFold, cross_val_score, train_test_split

In [ ]:
drive.mount('/content/drive')
warnings.filterwarnings("ignore")
TEST_SIZE = 0.2
RANDOM_STATE = 42

Mounted at /content/drive


## EDA

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DA&ML (lab2)/train.csv")

In [ ]:
df = df.sample(frac=0.1, random_state=RANDOM_STATE)

In [ ]:
df.head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
4974990,10,457,50,649143.74,1,0.999808,1.311118e+07,NaN,NaN,0.999626,45605.28,0.999808,23877.15,0.999746,4.270077,25140,457_50_10
2690154,197,249,70,2480676.30,1,0.998133,7.039544e+06,NaN,NaN,0.997941,51840.00,0.998229,46462.08,0.998093,-3.619790,13702,249_70_197
2781738,41,257,280,95485320.75,1,1.001244,3.183543e+08,NaN,NaN,1.001466,8858.71,1.001742,4340.16,1.001651,-2.250075,14163,257_280_41
4476578,198,411,420,2487847.57,1,1.001232,1.112020e+08,1.007072,1.003735,1.000954,543017.12,1.001232,212868.00,1.001153,-1.940131,22647,411_420_198
1208823,104,113,30,7020381.95,-1,0.999986,1.959290e+07,NaN,NaN,0.999986,5387.00,1.000172,34267.68,1.000011,1.289845,6218,113_30_104


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 523798 entries, 4974990 to 4743503
Data columns (total 17 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   stock_id                 523798 non-null  int64  
 1   date_id                  523798 non-null  int64  
 2   seconds_in_bucket        523798 non-null  int64  
 3   imbalance_size           523781 non-null  float64
 4   imbalance_buy_sell_flag  523798 non-null  int64  
 5   reference_price          523781 non-null  float64
 6   matched_size             523781 non-null  float64
 7   far_price                234817 non-null  float64
 8   near_price               238560 non-null  float64
 9   bid_price                523781 non-null  float64
 10  bid_size                 523798 non-null  float64
 11  ask_price                523781 non-null  float64
 12  ask_size                 523798 non-null  float64
 13  wap                      523781 non-null  float64
 1

In [ ]:
df.describe()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id
count,523798.000000,523798.000000,523798.000000,5.237810e+05,523798.000000,523781.000000,5.237810e+05,234817.000000,238560.000000,523781.000000,5.237980e+05,523781.000000,5.237980e+05,523781.000000,523790.000000,523798.000000
mean,99.338779,241.503505,270.270677,5.725091e+06,-0.013677,0.999993,4.517591e+07,1.000349,0.999643,0.999724,5.193021e+04,1.000262,5.367664e+04,0.999990,-0.031147,13309.719852
std,57.857558,138.561598,158.727198,2.051634e+07,0.885420,0.002533,1.382826e+08,0.222864,0.012240,0.002500,1.143823e+05,0.002510,1.297909e+05,0.002497,9.452502,7620.904598
min,0.000000,0.000000,0.000000,0.000000e+00,-1.000000,0.952713,4.316610e+03,0.000245,0.881574,0.952747,0.000000e+00,0.953323,0.000000e+00,0.952752,-281.400100,0.000000
25%,49.000000,122.000000,130.000000,8.477945e+04,-1.000000,0.998761,5.291234e+06,0.996332,0.997097,0.998526,7.396658e+03,0.999026,7.837640e+03,0.998777,-4.550219,6729.000000
50%,99.000000,242.000000,270.000000,1.114721e+06,0.000000,0.999965,1.287701e+07,0.999874,0.999884,0.999727,2.194293e+04,1.000207,2.300788e+04,0.999997,-0.069737,13349.000000
75%,150.000000,361.000000,410.000000,4.214957e+06,1.000000,1.001173,3.280247e+07,1.003290,1.002575,1.000903,5.600865e+04,1.001412,5.800546e+04,1.001149,4.400015,19904.000000
max,199.000000,480.000000,540.000000,2.953795e+09,1.000000,1.052420,7.705220e+09,103.937590,1.309732,1.048628,2.707694e+07,1.053535,2.896656e+07,1.049047,241.220000,26454.000000


In [ ]:
df.isnull().sum() / df.shape[0] * 100

stock_id                    0.000000
date_id                     0.000000
seconds_in_bucket           0.000000
imbalance_size              0.003246
imbalance_buy_sell_flag     0.000000
reference_price             0.003246
matched_size                0.003246
far_price                  55.170314
near_price                 54.455725
bid_price                   0.003246
bid_size                    0.000000
ask_price                   0.003246
ask_size                    0.000000
wap                         0.003246
target                      0.001527
time_id                     0.000000
row_id                      0.000000
dtype: float64

In [ ]:
df.isnull().sum()

stock_id                        0
date_id                         0
seconds_in_bucket               0
imbalance_size                 17
imbalance_buy_sell_flag         0
reference_price                17
matched_size                   17
far_price                  288981
near_price                 285238
bid_price                      17
bid_size                        0
ask_price                      17
ask_size                        0
wap                            17
target                          8
time_id                         0
row_id                          0
dtype: int64

In [ ]:
df.columns

Index(['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap', 'target', 'time_id', 'row_id'],
      dtype='object')

In [ ]:
df.dropna(subset=["target"], inplace=True)
df.dropna(how="all", inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
features = [col for col in df.columns if col not in ["row_id", "target"]]
print(features)

['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size', 'imbalance_buy_sell_flag', 'reference_price', 'matched_size', 'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price', 'ask_size', 'wap', 'time_id']


In [ ]:
X = df[features].copy(deep=True)
y = df["target"].copy(deep=True)

In [ ]:
X.fillna(X.median(), inplace=True)

## Feature engineering

In [ ]:
# Function to compute triplet imbalance in parallel using Numba
@njit(parallel=True)
def compute_triplet_imbalance(df_values, comb_indices):
    num_rows = df_values.shape[0]
    num_combinations = len(comb_indices)
    imbalance_features = np.empty((num_rows, num_combinations))

    # Loop through all combinations of triplets
    for i in prange(num_combinations):
        a, b, c = comb_indices[i]

        # Loop through rows of the DataFrame
        for j in range(num_rows):
            max_val = max(df_values[j, a], df_values[j, b], df_values[j, c])
            min_val = min(df_values[j, a], df_values[j, b], df_values[j, c])
            mid_val = df_values[j, a] + df_values[j, b] + df_values[j, c] - min_val - max_val

            # Prevent division by zero
            if mid_val == min_val:
                imbalance_features[j, i] = np.nan
            else:
                imbalance_features[j, i] = (max_val - mid_val) / (mid_val - min_val)

    return imbalance_features


# Function to calculate triplet imbalance for given price data and a DataFrame
def calculate_triplet_imbalance_numba(price, df):
    # Convert DataFrame to numpy array for Numba compatibility
    df_values = df[price].values
    comb_indices = [(price.index(a), price.index(b), price.index(c)) for a, b, c in combinations(price, 3)]

    # Calculate the triplet imbalance using the Numba-optimized function
    features_array = compute_triplet_imbalance(df_values, comb_indices)

    # Create a DataFrame from the results
    columns = [f"{a}_{b}_{c}_imb2" for a, b, c in combinations(price, 3)]
    features = pd.DataFrame(features_array, columns=columns)

    return features


def numba_imb_features(df):
    prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
    sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]

    for func in ["mean", "std", "skew", "kurt"]:
        df[f"all_prices_{func}"] = df[prices].agg(func, axis=1)
        df[f"all_sizes_{func}"] = df[sizes].agg(func, axis=1)

    # Calculate triplet imbalance features using the Numba-optimized function
    for c in [['ask_price', 'bid_price', 'wap', 'reference_price'], sizes]:
        triplet_feature = calculate_triplet_imbalance_numba(c, df)
        df[triplet_feature.columns] = triplet_feature.values
    return df

In [ ]:
def feature_engineering(X):

    _X = X.copy()

    # features taken from https://www.kaggle.com/code/zulqarnainali/explained-singel-model-optiver
    # market_urgency', 'seconds_in_bucket', 'liquidity_imbalance', 'imbalance_momentum', 'price_spread', 'matched_imbalance', 'bid_size', 'matched_size', 'spread_intensity', 'ask_size'
    _X["volume"] = _X["ask_size"] + _X["bid_size"]
    _X["mid_price"] = (_X["ask_price"] + _X["bid_price"]) / 2
    _X["liquidity_imbalance"] = (_X["bid_size"] - _X["ask_size"]) / (_X["bid_size"] + _X["ask_size"])
    _X["matched_imbalance"] = (_X["imbalance_size"] - _X["matched_size"]) / (_X["matched_size"] + _X["imbalance_size"])
    _X["size_imbalance"] = _X["bid_size"] / _X["ask_size"]

    _X["imbalance_momentum"] = _X.groupby(['stock_id'])['imbalance_size'].diff(periods=1) / _X['matched_size']
    _X["price_spread"] = _X["ask_price"] - _X["bid_price"]
    _X["spread_intensity"] = _X.groupby(['stock_id'])['price_spread'].diff()
    _X['price_pressure'] = _X['imbalance_size'] * (_X['ask_price'] - _X['bid_price'])
    _X['market_urgency'] = _X['price_spread'] * _X['liquidity_imbalance']
    _X['depth_pressure'] = (_X['ask_size'] - _X['bid_size']) * (_X['far_price'] - _X['near_price'])

    _X["dow"] = _X["date_id"] % 5  # Day of the week
    _X["seconds"] = _X["seconds_in_bucket"] % 60  # Seconds
    _X["minute"] = _X["seconds_in_bucket"] // 60  # Minutes

    # # Calculate shifted and return features for specific columns
    for col in ['matched_size', 'imbalance_size', 'reference_price']:
        for window in [1, 2, 3, 10]:
            _X[f"{col}_shift_{window}"] = _X.groupby('stock_id')[col].shift(window)
            _X[f"{col}_ret_{window}"] = _X.groupby('stock_id')[col].pct_change(window)

    # # Calculate diff features for specific columns
    for col in ['ask_price', 'bid_price', 'ask_size', 'bid_size']:
        for window in [1, 2, 3, 10]:
            _X[f"{col}_diff_{window}"] = _X.groupby("stock_id")[col].diff(window)

    # Create features for pairwise price imbalances
    prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
    for c in combinations(prices, 2):
        _X[f"{c[0]}_{c[1]}_imb"] = _X.eval(f"({c[0]} - {c[1]})/({c[0]} + {c[1]})")

    # Generate imbalance features
    _X = numba_imb_features(_X)

    _X = _X.replace([np.inf, -np.inf], 0)
    _X.drop(columns=['date_id'], inplace=True)

    return _X

## Train/test split

In [ ]:
%%time

number_of_days = len(X.date_id.unique()) # There are 481 trading days, we'll take the last n as testing set
testing_days = 10
training_days = number_of_days - testing_days
print(training_days, testing_days)

# mask to grab the days for training and testing
training_mask = X.date_id <= training_days
testing_mask = X.date_id > training_days

# subset and make training and validation sets
X_train, X_val, y_train, y_val = X[training_mask], X[testing_mask], y[training_mask], y[testing_mask]
X_train.shape, X_val.shape, y_train.shape, y_val.shape

# apply feature engineering
X_train = feature_engineering(X_train)
X_val = feature_engineering(X_val)

# Convert all numerics to float32 to reduce memory footprint
X_train = X_train.astype(np.float32)
X_val = X_val.astype(np.float32)

print(X_train.shape, X_val.shape)

471 10
(513890, 99) (9900, 99)
CPU times: user 6.16 s, sys: 2.85 s, total: 9.01 s
Wall time: 9.32 s


In [ ]:
X_train.fillna(X_train.median(), inplace=True)
X_val.fillna(X_val.median(), inplace=True)

## RandomForestRegressor + Optuna

In [ ]:
def objective(trial, X, y, groups):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 10, 50),
        "max_depth": trial.suggest_int("max_depth", 5, 30),
        "min_samples_split": trial.suggest_int("min_samples_split", 1e3, 1e5),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1e3, 1e5),
    }

    model = RandomForestRegressor(**params)
    cv = GroupKFold(n_splits=5)

    scores = cross_val_score(model, X, y, cv=cv, groups=groups, scoring="neg_mean_absolute_error", n_jobs=8, verbose=1)
    score = -scores.mean()

    return score

In [ ]:
sampler = TPESampler(seed=RANDOM_STATE)
study = optuna.create_study(direction="maximize", sampler=sampler)

[I 2023-12-19 20:55:51,309] A new study created in memory with name: no-name-9932bb36-490f-43f5-9e5e-69fbeee60945


In [ ]:
%%time
study.optimize(lambda trial: objective(trial, X_train, y_train, X_train["time_id"]), n_trials=5)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   5 | elapsed: 10.9min remaining: 16.4min
[Parallel(n_jobs=8)]: Done   5 out of   5 | elapsed: 11.0min finished
[I 2023-12-19 21:06:52,923] Trial 0 finished with value: 6.3723308926138476 and parameters: {'n_estimators': 25, 'max_depth': 29, 'min_samples_split': 73468, 'min_samples_leaf': 60267}. Best is trial 0 with value: 6.3723308926138476.
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   5 | elapsed:  4.0min remaining:  6.1min
[Parallel(n_jobs=8)]: Done   5 out of   5 | elapsed:  4.1min finished
[I 2023-12-19 21:10:56,640] Trial 1 finished with value: 6.387215339246798 and parameters: {'n_estimators': 16, 'max_depth': 9, 'min_samples_split': 6750, 'min_samples_leaf': 86752}. Best is trial 1 with value: 6.387215339246798.
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n

CPU times: user 46 s, sys: 10.5 s, total: 56.6 s
Wall time: 1h 30min 38s


In [ ]:
best_params = study.best_params
print(f"Best Hyperparameters: {best_params}")
# Best Hyperparameters: {'n_estimators': 34, 'max_depth': 23, 'min_samples_split': 3037, 'min_samples_leaf': 97022}

Best Hyperparameters: {'n_estimators': 34, 'max_depth': 23, 'min_samples_split': 3037, 'min_samples_leaf': 97022}


In [ ]:
rfr = RandomForestRegressor(**best_params)
rfr.fit(X_train, y_train)
preds = rfr.predict(X_val)
rfr_mae = mean_absolute_error(y_val, preds)
print(f"MAE for RFR: {rfr_mae}") # 5.6627036275540785

MAE for RFR: 5.6627036275540785


## LGBMRegressor + Optuna

In [ ]:
def objective_lgbm(trial, X, y, groups):
    params = {
        "num_leaves": trial.suggest_int("num_leaves", 10, 100),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
        "max_depth": trial.suggest_int("max_depth", 5, 30)
    }

    model = lgb.LGBMRegressor(**params)
    cv = GroupKFold(n_splits=5)
    scores = cross_val_score(model, X, y, cv=cv, groups=groups, scoring="neg_mean_absolute_error", n_jobs=8, verbose=1)
    score = -scores.mean()

    return score

In [ ]:
sampler_lgbm = TPESampler(seed=RANDOM_STATE)
study_lgbm = optuna.create_study(direction="maximize", sampler=sampler_lgbm)

[I 2023-12-19 22:29:21,197] A new study created in memory with name: no-name-d5e4e14a-740c-4c3e-88c7-b784d0cb499b


In [ ]:
%%time
study_lgbm.optimize(lambda trial: objective_lgbm(trial, X_train, y_train, X_train["time_id"]), n_trials=5)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   5 | elapsed:  1.8min remaining:  2.7min
[Parallel(n_jobs=8)]: Done   5 out of   5 | elapsed:  1.9min finished
[I 2023-12-19 22:31:13,385] Trial 0 finished with value: 6.308050263414759 and parameters: {'num_leaves': 44, 'learning_rate': 0.09556428757689246, 'max_depth': 24}. Best is trial 0 with value: 6.308050263414759.
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   5 | elapsed:  2.4min remaining:  3.6min
[Parallel(n_jobs=8)]: Done   5 out of   5 | elapsed:  2.4min finished
[I 2023-12-19 22:33:36,333] Trial 1 finished with value: 6.314468366248745 and parameters: {'num_leaves': 64, 'learning_rate': 0.02404167763981929, 'max_depth': 9}. Best is trial 1 with value: 6.314468366248745.
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   5 | elapse

CPU times: user 8.44 s, sys: 1.12 s, total: 9.56 s
Wall time: 10min 37s


In [ ]:
best_params_lgbm = study_lgbm.best_params
print(f"Best Hyperparameters: {best_params_lgbm}")
#Best Hyperparameters: {'num_leaves': 74, 'learning_rate': 0.011852604486622221, 'max_depth': 30}

Best Hyperparameters: {'num_leaves': 74, 'learning_rate': 0.011852604486622221, 'max_depth': 30}


In [ ]:
lgbmr = lgb.LGBMRegressor(**best_params_lgbm)
lgbmr.fit(X_train, y_train)
preds_lgbm = lgbmr.predict(X_val)
lgbmr_mae = mean_absolute_error(y_val, preds_lgbm)
print(f"MAE for LGBMR: {lgbmr_mae}") # 5.6299824722818395

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.634310 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 23993
[LightGBM] [Info] Number of data points in the train set: 513890, number of used features: 99
[LightGBM] [Info] Start training from score -0.028599
MAE for LGBMR: 5.6299824722818395


## Stacking

In [ ]:
df_preds = pd.DataFrame({"RFR_preds": preds, "LGBMR_preds": preds_lgbm})
df_preds.reset_index(drop=True, inplace=True)
y_val.reset_index(drop=True, inplace=True)
df_meta = pd.concat([df_preds, y_val], axis=1)

X_meta = df_meta.drop("target", axis=1).copy(deep=True)
y_meta = df_meta["target"].copy(deep=True)


X_train_meta, X_test_meta, y_train_meta, y_test_meta = train_test_split(
    X_meta, y_meta, test_size=TEST_SIZE, random_state=RANDOM_STATE)


meta_model = LinearRegression()
meta_model.fit(X_train_meta, y_train_meta)
preds_meta = meta_model.predict(X_test_meta)

mae_stacking = mean_absolute_error(y_test_meta, preds_meta)
print(f"MAE for Stacking: {mae_stacking}") # 5.739354193297307

MAE for Stacking: 5.739354193297307
